In [1]:
from typing import Dict, List
import matplotlib.pyplot as plt
import networkx as nx
import overpy
import numpy as np
from operator import itemgetter

In [13]:
api = overpy.Overpass()
result = api.query("way(24064204); out id;")
result.ways

OverpassBadRequest: Error: line 1: parse error: Invalid parameter for print: &quot;id&quot; 

In [17]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [18]:
from scripts.config import Config

In [19]:
def eval_way(way, config: Config = Config()):
    score = 0
    max_score = config.weight_sum()
    for tag, value in way.tags.items():
        weight, mapping = config[tag]
        score += weight * mapping.get(value, 0)
    return score / max_score

In [20]:
def filter_ways(result, threshold):
    ways = []
    for way in result.ways:
        score = eval_way(way)
        if score > threshold:
            ways.append(way)
    return ways

In [21]:
def node_in_area(node: overpy.Node, config: Config):
    south, west, north, east = config.bounding_box.South, config.bounding_box.West, \
        config.bounding_box.North, config.bounding_box.East
    lon, lat = float(node.lon), float(node.lat)
    return (lon <= east) and (lon >= west) and (lat >= south) and (lat <= north)

In [22]:
def ways_to_edges(ways: List[overpy.Way], config: Config = Config()):
    link_counter: Dict[str, int] = {}
    for way in ways:
        nodes = way.get_nodes(resolve_missing=True)
        for node in nodes:
            link_counter[node.id] = link_counter.get(node.id, 0) + 1

    edges = []
    all_nodes = set()
    for way in ways:
        nodes = way.get_nodes(resolve_missing=True)
        nodes = [node for node in nodes if node_in_area(node, config)]
        if len(nodes) == 2:
            edges.append([nodes[0].id, nodes[1].id])  # add way as an edge
            all_nodes.add(nodes[0])
            all_nodes.add(nodes[1])
            continue
        head = nodes[0]
        tail = nodes[len(nodes) - 1]
        prev = head
        for i in range(1, len(nodes)):
            node = nodes[i]
            if (link_counter[node.id] > 1 or node is tail) and prev.id != node.id:
                edges.append([prev.id, node.id])
                all_nodes.add(prev)
                all_nodes.add(node)
                prev = node
    return edges, list(all_nodes)

In [23]:
api = overpy.Overpass()
result = api.query("nwr(56.3284, -2.8350, 56.3437, -2.7855); out;")  # st andrews
# result = api.query("nwr(56.3375, -2.8059, 56.3422, -2.7888); out;")
target_way_query = api.query("way(24064204); out;")
target_way = target_way_query.ways[0]

In [24]:
ways = filter_ways(result, 0.15)

In [25]:
ways_ids = [way.id for way in ways]

In [26]:
24064204 in ways_ids

False

In [ ]:
link_counter: Dict[str, int] = {}
for way in ways:
    nodes = way.get_nodes(resolve_missing=True)
    if way.id == 24064204:
        print(nodes)
    for node in nodes:
        link_counter[node.id] = link_counter.get(node.id, 0) + 1

In [ ]:
link_counter

In [ ]:
way = target_way
way.get_nodes(resolve_missing=True)

In [ ]:
edges = []
all_nodes = set()

nodes = way.get_nodes(resolve_missing=True)
nodes = [node for node in nodes if node_in_area(node, Config())]
nodes

In [ ]:
head = nodes[0]
head

In [ ]:
tail = nodes[len(nodes) - 1]
tail

In [ ]:
prev = head
for i in range(1, len(nodes)):
    node = nodes[i]
    if (link_counter.get(node.id, 0) > 1 or node is tail) and prev.id != node.id:
        edges.append([prev.id, node.id])
        all_nodes.add(prev)
        all_nodes.add(node)
        prev = node

In [ ]:
edges

In [ ]:
edges, all_nodes = ways_to_edges(ways)

In [ ]:
ids = set([node.id for node in all_nodes])
print(261732339 in ids)

In [ ]:
target = 261732339
for edge in edges:
    if edge[0] == edge[1]:
        print(edge)

In [ ]:
layout = {node.id: (float(node.lon), float(node.lat)) for node in all_nodes}

In [ ]:
G = nx.Graph(edges)

In [ ]:
nx.draw_networkx(G, pos=layout, with_labels=False, node_size=10, font_size=6)

In [ ]:
def poly_area(x,y):
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

In [ ]:
def group_area(group, layout):
    coordinates = [layout.get(node_id) for node_id in group]
    south = min(coordinates, key=lambda x: x[1])
    west = min(coordinates, key=lambda x: x[0])
    north = max(coordinates, key=lambda x: x[1])
    east = max(coordinates, key=lambda x: x[0])
    bbox = [south, west, north, east]
    return poly_area([n[0] for n in bbox], [n[1] for n in bbox])

In [ ]:
sorted_groups = sorted(nx.connected_components(G), key=lambda g: group_area(g, layout), reverse=True)
len(sorted_groups[0])

In [ ]:
largest_component = G.subgraph(list(sorted_groups[0]))
second_component = G.subgraph(list(sorted_groups[1]))
nx.draw_networkx(G, pos=layout, with_labels=False, \
                 node_size=5)
nx.draw_networkx(largest_component, pos=layout, \
                 with_labels=False, node_size=5, \
                 node_color='r', edge_color='r', font_size=6)
nx.draw_networkx(second_component, pos=layout, \
                 with_labels=False, node_size=5, \
                 node_color='m', edge_color='m', font_size=6)